# Notebook for Testing different confidence bands

In [1]:
import numpy as np
import math
import sys
import os
import itertools
import time
from tqdm import tqdm
from joblib import Parallel, delayed
from IPython.display import display, HTML
from scipy import stats
from scipy.sparse import csr_matrix
from scipy import special
from numpy.linalg import cholesky
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu
src_path = os.path.abspath(os.path.join(os.getcwd(), "../src"))
sys.path.append(src_path)
from forest_v2 import RegressionTreeModel
from forest_v2 import RandomForestModel
from forest_v2 import HistogramEstimator
import functions as fcts

In [2]:
# dimension of the feature space
p = 4

#regression function
regression_fct=fcts.m_p4_01

# betas for confidence levels 1-\beta
beta=np.array([0.1,0.05,0.01])

## Covariance estimation

In [3]:
# depth of the trees
k = 5 #5

# smaller k for evaluation grid (choice justified later)
k2 = 3 #3

#Ehrenfest parameters
B = 12
delta = 7
np.set_printoptions(suppress=True)

In [4]:
# estimation of all possible covariance entries using the parameters for the partition construction
np.random.seed(0)
n_parts=50000 # number of split pairs generated

# list of all combinations of closeness relations between two points in the feature space
comb_list=[]
for com in itertools.combinations_with_replacement(range(0, k + 1), p):
    comb_list.append(com)
combs=np.array(comb_list)

#simulation of splits according to the uniform CPRF and calculation of the average intersection volume for all combinations
cu_vol=np.zeros(len(combs))
for j in range(n_parts):
    s1=np.bincount(np.random.choice(p, k, replace=True),minlength=p)
    s2=np.bincount(np.random.choice(p, k, replace=True),minlength=p)
    for i in range(len(combs)):
        cu_vol[i]+=fcts.vol_intersec_2(combs[i],s1,s2)
av_vol=cu_vol/n_parts

# calculation of estimated covariance from average volumina
V_cap_uni=av_vol[len(av_vol)-1]
cov_entries_uni=av_vol/V_cap_uni

#simulation of splits according to the Ehrenfest CPRF and calculation of the average intersection volume for all combinations
cu_vol=np.zeros(len(combs))
for j in range(n_parts):
    s1=fcts.ehr_splits(np.zeros(p),np.ones(p)*B,delta,k)
    s2=fcts.ehr_splits(np.zeros(p),np.ones(p)*B,delta,k)
    for i in range(len(combs)):
        cu_vol[i]+=fcts.vol_intersec_2(combs[i],s1,s2)
av_vol=cu_vol/n_parts

# calculation of estimated covariance from average volumina
V_cap_ehr=av_vol[len(av_vol)-1]
cov_entries_ehr=av_vol/V_cap_ehr

print("Uniform V_cap:", V_cap_uni,", Ehrenfest V_cap:", V_cap_ehr,", 2^-k:", 1/2**k,", 2^-2k:",1/2**(2*k))

#cov_entries_ehr, cov_entries_uni, #len(cov_entries_uni),special.binom(k+p,p)

Uniform V_cap: 0.00907939453125 , Ehrenfest V_cap: 0.009599375 , 2^-k: 0.03125 , 2^-2k: 0.0009765625


Justification for the choice of $k_2$. For any pair of points in the feature space with closeness $(k_2,k_2)$ or larger, the Gaussian process' covariance should be close to one.

In [5]:
min_prob=0.99 # 0.99
print("Smallest closeness with cov > min_prob, Ehr.:",comb_list[np.argmax(cov_entries_ehr>min_prob)], "and Uni:", comb_list[np.argmax(cov_entries_uni>min_prob)])
print("Corresponding covariances: ", cov_entries_ehr[np.argmax(cov_entries_ehr>min_prob)], "and",cov_entries_uni[np.argmax(cov_entries_uni>min_prob)])

Smallest closeness with cov > min_prob, Ehr.: (3, 3, 3, 3) and Uni: (3, 3, 3, 3)
Corresponding covariances:  0.9989582655120777 and 0.9980725586998376


In [6]:
# Filling in the covariance entries in the matrices
X_test=fcts.test_grid(p,k2)
start_time = time.time()
X_test_M=np.array(list(itertools.product(X_test, repeat=2)))#.reshape((2**(k2*p),2**(k2*p),2,2))
X_test_V=X_test_M.reshape(2**(k2*p*2),2*p)
t_values=np.arange(0,k+1)
v_t=(2**t_values).reshape(-1,1)
fin_com_cell=np.empty((2**(k2*p*2),p))
for i in range(p):
    X_temp_1=X_test_V[:,i].reshape(-1,1)
    X_temp_2=X_test_V[:,i+p].reshape(-1,1)
    ints1=np.dot(X_temp_1,v_t.T).astype(int)
    ints2=np.dot(X_temp_2,v_t.T).astype(int)
    equal_mask=ints1==ints2
    fin_com_cell[:,i] = np.max(np.where(equal_mask, t_values, -np.inf),axis=1)
M=fin_com_cell.astype(int)
M=np.sort(M,axis=1)
M_int=np.sum(M*10**np.arange(p-1,-1,-1),axis=1)
int_combs=np.sum(combs*10**np.arange(p-1,-1,-1),axis=1)
indices = np.searchsorted(int_combs, M_int)
cov_uni=cov_entries_uni[indices].reshape((2**(k2*p),2**(k2*p)))
cov_ehr=cov_entries_ehr[indices].reshape((2**(k2*p),2**(k2*p)))
end_time = time.time()
print(f"Laufzeit: {end_time - start_time} Sekunden")
print("Size of the cov matrix: ",cov_uni.shape) #, np.info(cov_uni), np.info(cov_ehr)

Laufzeit: 25.83875870704651 Sekunden
Size of the cov matrix:  (4096, 4096)


The next two cells perform the Cholesky decomposition of the covariance matrices. If this is not possible due to the estimation error, they reconstruct the matrix beforehand. Either based on all eigenvalues larger than $\epsilon$ and their corresponding eigenvectors or based on the SVD. 

In [24]:
epsilon=1e-6
try:
    L_uni = cholesky(cov_uni+epsilon * np.eye(2**(k2*p))) #+1e-3 * np.eye(2**(k*p))
except Exception as e:
    eig_values, eig_vectors = np.linalg.eigh(cov_uni)
    corrected_eig_values = np.maximum(eig_values, epsilon)
    fixed_cov_uni = eig_vectors @ np.diag(corrected_eig_values) @ eig_vectors.T
    L_uni = cholesky(fixed_cov_uni)
    print(f"An error occured: {e}.", "We used reconstruction of the covariance matrix via eigenvectors before the cholesky decomposition.")
else:
    print("We used the cholesky decomposition of the original covariance matrix.")  
#stats.describe((L_uni @ L_uni.T)[:,0] -cov_uni[:,0])

An error occured: Matrix is not positive definite. We used reconstruction of the covariance matrix via eigenvectors before the cholesky decomposition.


In [25]:
epsilon=1e-6
try:
    L_ehr = cholesky(cov_ehr+epsilon * np.eye(2**(k2*p))) #+1e-3 * np.eye(2**(k*p))
except Exception as e:
    eig_values, eig_vectors = np.linalg.eigh(cov_ehr)
    corrected_eig_values = np.maximum(eig_values, epsilon)
    fixed_cov_ehr = eig_vectors @ np.diag(corrected_eig_values) @ eig_vectors.T
    L_ehr = cholesky(fixed_cov_ehr)
    print(f"An error occured: {e}.", "We used reconstruction of the covariance matrix via eigenvectors before the cholesky decomposition.")
else:
    print("We used the cholesky decomposition of the original covariance matrix.")
#stats.describe((L_ehr @ L_ehr.T)[:,0] -cov_ehr[:,0])

An error occured: Matrix is not positive definite. We used reconstruction of the covariance matrix via eigenvectors before the cholesky decomposition.


## Quantile estimation of the Gaussian processes

In [37]:
#Generating suprema of the Gaussian process to estimate the quantiles
np.random.seed(0)
sups_GP_uni=[]
sups_GP_ehr=[]
# number of suprema
n_sups=100000
for _ in tqdm(range(n_sups)):
    e=np.random.normal(0,1,2**(k2*p))
    gp_uni = L_uni @ e
    gp_ehr = L_ehr @ e
    sups_GP_uni.append(np.max(np.abs(gp_uni)))
    sups_GP_ehr.append(np.max(np.abs(gp_ehr)))

100%|██████████████████████████████████████████████████████████████████████████| 100000/100000 [23:24<00:00, 71.21it/s]


In [38]:
#empirical quantiles
quants_uni=np.quantile(sups_GP_uni,1-beta)
quants_ehr=np.quantile(sups_GP_ehr,1-beta)

#stats.describe(sups_GP_uni), stats.describe(sups_GP_ehr)
print("Quantiles for k = ",k," at:   ",1-beta)
print("Uniform CPRF:   ",quants_uni)
print("Ehrenfest CPRF: ", quants_ehr)

Quantiles for k =  5  at:    [0.9  0.95 0.99]
Uniform CPRF:    [3.65631766 3.86412559 4.29084145]
Ehrenfest CPRF:  [3.61200856 3.82227777 4.25334828]


# CB Tests

In [39]:
# if necessary create a text file for the results of the simulations
if not os.path.isfile('Simulation results.txt'):
    with open('Simulation results.txt', 'w') as f:
        f.write( "Simulation results of asymptotic confidence bands"+ '\n')
    print("The file 'Simulation results.txt' was created.")
else:
    print("The file 'Simulation results.txt' already existed.")

The file 'Simulation results.txt' already existed.


Set the parameters:

In [40]:
# training sample size
n_samples = 2000 #4000

#factor for subsample size
r = 0.75 

# bandwidth for estimation of sigma by kernel estimator
h_g=1/n_samples**(1/2)

In [41]:
# asymptotic standard deviations (pointwise)
as_std_uni=np.sqrt(2**(2*k)*V_cap_uni/n_samples)
as_std_ehr=np.sqrt(2**(2*k)*V_cap_ehr/n_samples)

#confidence band radii based on the quantiles and the standard deviations
cb_rad_uni=as_std_uni*quants_uni
cb_rad_ehr=as_std_ehr*quants_ehr

print("Confidence band radii for n = ",n_samples," p = ", p, " and k = ",k, "with confidence levels :", 1-beta)
print("Uniform CPRF:   ",cb_rad_uni)
print("Ehrenfest CPRF: ", cb_rad_ehr)

Confidence band radii for n =  2000  p =  4  and k =  5 with confidence levels : [0.9  0.95 0.99]
Uniform CPRF:    [0.24929143 0.26345998 0.2925539 ]
Ehrenfest CPRF:  [0.25322421 0.26796539 0.29818611]


The next cell is for the simulation of asymptotic confidence bands for the two random forests.

In [42]:
def run_test(i, n_samples, p, m_factor, regression_fct, delta, B, error_dist, rand_seed, n_trees, k, r, X_test, m_true_grid, h_g):
    np.random.seed(rand_seed + i)  # Use a unique seed for each iteration
    data_rng = np.random.default_rng(rand_seed + i)  # Create a new random number generator for each iteration

    # Generate data
    e = error_dist.rvs(size=n_samples, random_state=data_rng)
    X = data_rng.random(size=(n_samples , p))

    m = m_factor * regression_fct(X)
    Y = m + e

    # Instantiate models
    model_uni = RandomForestModel(n_trees=n_trees, max_depth=k, sample_size_fct=r, tree_type="Uni")
    model_ehr = RandomForestModel(n_trees=n_trees, max_depth=k, sample_size_fct=r, tree_type="Ehr", delta=delta, B=B)

    # Train models
    model_uni.train(X, Y)
    model_ehr.train(X, Y)

    # Make predictions
    preds_uni = model_uni.predict(X_test)
    preds_ehr = model_ehr.predict(X_test)

    # Calculate errors
    error_uni = preds_uni - m_true_grid
    error_ehr = preds_ehr - m_true_grid

    sup_uni = np.max(np.abs(error_uni))
    sup_ehr = np.max(np.abs(error_ehr))

    sigma_hat = np.sqrt(fcts.sigma_hat_gauss(X, Y, h_g))

    return sup_uni, sup_ehr, sigma_hat

In [43]:
#grid for supremum calculation
eps=1/2**20
g=2**k2
splits=np.linspace(0, 1, num=g, endpoint=False)
xl=splits+eps
xr=splits+1/g-eps
axis_grid=np.sort(np.concatenate((xl,xr)))
prod=list(itertools.product(axis_grid, repeat=p))
grid=np.array(prod)
grid.shape

(65536, 4)

In [47]:
# parameters to vary:

# number of cpus used
n_cpus=2

# number of confidence bands
n_tests=1000

# random seed for both data and tree construction (two generators for replicable results)
rand_seed=12345

# vector of error std, used: [0.75,1,1.25]
sigs=[1]#0.5,0.75,1,1.25]

# vector of error distributions, possible entries: 'norm', 'uni, 'tdx' where x is a place holder for the degrees of freedom
distributions=['norm']#,'uni','td4','td6'] 

# vector with number of trees
n_trees_vec=[100,50]#,100] 

# multiplier to in/exclude the regression function
# 1 for the normal regression model, 0 to set m=0 to test the asymptotic distribution without approximation error
m_factor=0

#bandwidth for the variance estimator
h_g=1/n_samples**(1/2) 

#_____________________________________________________________________________________________________

X_test=grid 
m_true_grid=m_factor*regression_fct(X_test)

#Simulation run
for sigma in sigs:
    for dist_name in distributions:
        if dist_name == "norm":
            error_dist=stats.norm(loc=0, scale=sigma)
        elif dist_name == "uni":
            error_dist=stats.uniform(loc=-0.5*sigma*np.sqrt(12), scale=sigma*np.sqrt(12))
        elif dist_name[:2]=="td":
            try:
                df=int(dist_name[2:len(dist_name)])
            except Exception as e:
                print(f"An error occured: {e}.", "Distribution not known. Skipped to next distribution.")
                continue
            error_dist=stats.t(df=df,loc=0,scale = sigma* np.sqrt((df-2)/df))
        else:
            print("Distribution not known. Skipped to next distribution.")
            continue
        
        for n_trees in n_trees_vec:
           
            sups_uni=[]
            sups_ehr=[]
            sigma_hats=[]         

            print("Simulation progress for error distribution "+dist_name+" with sigma =",sigma," and n_trees =",n_trees, ":")
            
            results = Parallel(n_jobs=n_cpus)(
                delayed(run_test)(i, n_samples, p, m_factor, regression_fct, delta, B, error_dist, rand_seed, n_trees, k, r, X_test, m_true_grid, h_g)
                for i in tqdm(range(n_tests))
            )
            
            # extract results
            for sup_uni, sup_ehr, sigma_hat in results:
                sups_uni.append(sup_uni)
                sups_ehr.append(sup_ehr)
                sigma_hats.append(sigma_hat)
            
            cover_num_uni=np.zeros(3)
            cover_num_ehr=np.zeros(3)
            for j in range(len(cb_rad_uni)):
                cover_num_uni[j]=sum(np.array(sups_uni)<np.array(sigma_hats)*cb_rad_uni[j])
                cover_num_ehr[j]=sum(np.array(sups_ehr)<np.array(sigma_hats)*cb_rad_ehr[j])
            n_CB=len(sups_uni)
            
            avg_cb_rad_uni=cb_rad_uni*np.mean(sigma_hats)
            avg_cb_rad_ehr=cb_rad_ehr*np.mean(sigma_hats)

            if m_factor==1:
                m_info = " and m = "+regression_fct.__name__
            elif m_factor ==0:
                m_info = " and m set to zero."
            else:
                m_info = " and m = "+str(m_factor)+"*"+regression_fct.__name__
            
            result_txt = ["","","Results  at "+time.ctime(),
                          "for Random Seed = "+str(rand_seed),
                          "and Regression model with:",
                          "p = "+str(p)+m_info,
                          "Parameters:",
                          "Sample size: "+str(n_samples),
                          "k="+str(k),
                          "r="+str(r*n_samples)]
            if error_dist.dist.name == 't':
                result_txt.append("Error distribution: t-Distribution with "+str(df)+" degrees of freedom")
            else:
                result_txt.append("Error distribution: " +str(error_dist.dist.name))
            
            result_txt.append("Error std: "+str(sigma))
            result_txt.append("Number of trees: "+str(n_trees))
            result_txt.append("Number of CBS: "+str(n_CB))
            result_txt+=["","Empirical Coverage for confidence bands with theoretical coverage "+ str(1-beta)+":"]
            result_txt.append("Uniform CPRF, number: "+str(cover_num_uni)+", percentage: "+ str(cover_num_uni/n_CB))
            result_txt.append("Ehrenfest CPRF, number: "+str(cover_num_ehr)+", percentage: "+ str(cover_num_ehr/n_CB))
            result_txt+=["","Average confidence band radius for theoretical coverage "+ str(1-beta)+":"]
            result_txt.append("Uniform CPRF: "+str(avg_cb_rad_uni))
            result_txt.append("Ehrenfest CPRF: "+str(avg_cb_rad_ehr))
            
            # save results of combination in txt file
            with open('Simulation results.txt', 'a') as f:
                for line in result_txt:
                    f.write(line + '\n')
            
            print("Simulation for distribution "+dist_name+" with sigma =",sigma," and n_trees =",n_trees, " complete.")

Simulation progress for error distribution norm with sigma = 1  and n_trees = 100 :


100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [17:07:22<00:00, 61.64s/it]


Simulation for distribution norm with sigma = 1  and n_trees = 100  complete.
Simulation progress for error distribution norm with sigma = 1  and n_trees = 50 :


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [8:33:10<00:00, 30.79s/it]


Simulation for distribution norm with sigma = 1  and n_trees = 50  complete.
